# Import necessary dependencies

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


!pip install optuna==2.3.0
!pip install transformers==4.1.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert


There are 1 GPU(s) available.
We will use the GPU: Tesla K80
Mon Jan 25 13:59:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    31W / 149W |     11MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                           

In [4]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

# Custom functions


In [20]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

DATA_COLUMN = "text"
LABEL_COLUMN = "label"
all_datasets = []

# Arabic Sentiment Dataset (Shared Task)

In [12]:
df_Custom = pd.read_csv("ASI_cleaned.csv")
df_Custom = df_Custom[["tweet","sentiment"]]  # we are interested in tweets and sentiment
df_Custom.columns = [DATA_COLUMN, LABEL_COLUMN]

print(df_Custom[LABEL_COLUMN].value_counts())

# df_Custom[LABEL_COLUMN] = df_Custom[LABEL_COLUMN].apply(lambda x: hard_map[x])
train_Custom, test_Custom = train_test_split(df_Custom, test_size=0.2, random_state=42)
label_list_Custom = ['NEG', 'POS', 'NEU']

data_Custom = Dataset("ASI_sentiment", train_Custom, test_Custom, label_list_Custom)
all_datasets.append(data_Custom)

NEU    5747
NEG    4618
POS    2179
Name: label, dtype: int64


# Arabic Sarcasm Dataset (Shared Task)

In [15]:
df_Custom = pd.read_csv("ASI_cleaned.csv")
df_Custom = df_Custom[["tweet","sarcasm"]]  # we are interested in tweets and sarcasm
df_Custom.columns = [DATA_COLUMN, LABEL_COLUMN]

print(df_Custom[LABEL_COLUMN].value_counts())

# df_Custom[LABEL_COLUMN] = df_Custom[LABEL_COLUMN].apply(lambda x: hard_map[x])
train_Custom, test_Custom = train_test_split(df_Custom, test_size=0.2, random_state=42)

label_list_Custom = ['FALSE', 'TRUE']

data_Custom = Dataset("ASI_sarcasm", train_Custom, test_Custom, label_list_Custom)
all_datasets.append(data_Custom)

False    10377
True      2167
Name: label, dtype: int64


# Arabic Hotel Reviews Dataset

In [23]:
df_Custom = pd.read_csv("ArSAS.csv")
df_Custom = df_Custom[["Tweet_text","Sentiment_label"]]  # we are interested in tweets and sarcasm
df_Custom.columns = [DATA_COLUMN, LABEL_COLUMN]

print(df_Custom[LABEL_COLUMN].value_counts())

# df_Custom[LABEL_COLUMN] = df_Custom[LABEL_COLUMN].apply(lambda x: hard_map[x])
train_Custom, test_Custom = train_test_split(df_Custom, test_size=0.2, random_state=42)

label_list_Custom = ['Positive', 'Negative', 'Neutral']

data_Custom = Dataset("Arabic_hotel_sentiment", train_Custom, test_Custom, label_list_Custom)
all_datasets.append(data_Custom)

Negative    7384
Neutral     6894
Positive    4400
Mixed       1219
Name: label, dtype: int64


#Trainer

In [ ]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna 

In [ ]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [16]:
for x in all_datasets:
  print("\nList of datasets in the memory: \n", x.name)


List of datasets in the memory: 
 custom_dataset

List of datasets in the memory: 
 custom_dataset

List of datasets in the memory: 
 ASI_sentiment

List of datasets in the memory: 
 ASI_sarcasm

List of datasets in the memory: 
 ASI_sarcasm

List of datasets in the memory: 
 ASI_sarcasm


Here you choose the AraBERT model and the dataset you want to evaluate. 
<br>
Be sure to check that max_len <= 2 * (average tweet length) 

In [18]:
dataset_name = 'ASI_sentiment'
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'classification'
max_len = 30

In [19]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


In [ ]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
selected_dataset.test[DATA_COLUMN]
selected_dataset.train.head()

,text,label
1584,RT aitnews تعتزم جوجل إطلاق الإصدار 55 من كروم...,NEU
2673,RT IhshrhI هجوم مجموعه كبيره من الناس في تايوا...,NEU
10146,هو يحبك وانتي مغرمة ب آخر والآخر يموت هياماآ ب...,NEG
10781,الشيخ عادل الكلباني 2,NEU
7464,إن كان من عادوا للتأسيسية يرون في هذا عملا وطن...,NEU


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [ ]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train[DATA_COLUMN].to_list(),selected_dataset.train[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(selected_dataset.test[DATA_COLUMN].to_list(),selected_dataset.test[LABEL_COLUMN].to_list(),model_name,max_len,label_map)

{'NEG': 0, 'POS': 1, 'NEU': 2}


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

# Training the AraBERT model 
(This can take some time)

This paert allows you to do a regular training with no hyper parameter optimization

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

313
2504


In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy
1,No log,0.690722,0.668548,0.630194,0.742576,0.659542,0.717417
2,0.696292,0.663761,0.739411,0.725005,0.732005,0.754499,0.753288
3,0.696292,0.678544,0.729489,0.713934,0.730691,0.731475,0.742527
4,0.384414,0.922025,0.733510,0.721414,0.726059,0.748757,0.744918
5,0.171220,1.214921,0.688678,0.655659,0.739687,0.670425,0.724990
6,0.171220,1.302205,0.725086,0.712404,0.720077,0.731875,0.737744
7,0.083689,1.501008,0.723602,0.706779,0.727657,0.720110,0.740534
8,0.043804,1.547198,0.718407,0.698568,0.730231,0.710364,0.739737


TrainOutput(global_step=2512, training_loss=0.27476883050837336)

In [ ]:
trainer.save_model("araBERT_sentiment_analyser")

NameError: ignored